In [4]:
import pandas as pd

# iplist = pd.read_csv('ip_blacklist_20160504.csv', sep=',', encoding='utf-8')
srlist = pd.read_csv('raw_data/test_data', sep=',', encoding='utf-8')

In [8]:
srcontents = srlist['내용']

In [7]:
from konlpy.tag import Twitter; t = Twitter()

In [24]:
df_srcontents = srcontents.to_frame()
df_srcontents

,내용
0,왜 싱글만 배송되었나요? 퀸은 언제오는지요.. 배송완료되지 않았는데.. 확인부탁드립니다.
1,취소하고싶은데취소가 안되내요 결제하고바로취소하려고눌럿는데 안눌려요 취소요청부탁드려요
2,연락이 한번도 없네요 ㅠㅠ 취소부탁드립니다취소 가능한가요?
3,현금영수증010 4506 8361발급해주세요~
4,저 이상품 매직딜로 결제 완료 된거맞는거죠? 예상 적립은 얼마이며 언제 적립되나요?...
5,이거 동생이 사달라고 해서 주문햇는데오늘 매장서 다른제품삿다고 주문취소해달라고 하네...
6,상품재질이 으깨어지면서 뭉침현상이 있습니다
7,"1월달 카드할인행사 정보를 알고싶은데요,~답변 부탁 드립니다"
8,결제카드 변경 하려는데 지금 국민카드 변경하면 7프로 혜택을 받을 수 있나요?
9,12월 16일 주문했는데 오늘 포장을 뜯어서 확인을 했어요..그레이로 주문을 했는데...


In [25]:
type(df_srcontents)

pandas.core.frame.DataFrame

In [28]:
def tokenize(doc):
    return ['/'.join(t) for t in t.pos(doc,norm=True,stem=True)]

In [31]:
df_srcontents.values

array([['왜 싱글만 배송되었나요? 퀸은 언제오는지요.. 배송완료되지 않았는데.. 확인부탁드립니다.'],
       ['취소하고싶은데취소가 안되내요  결제하고바로취소하려고눌럿는데 안눌려요 취소요청부탁드려요'],
       ['연락이 한번도 없네요 ㅠㅠ 취소부탁드립니다취소 가능한가요?'],
       ..., 
       [ '새 옷 안주머니에서 나온..겁니다.뭔..지..참...업체 관리..더 신경 쓰셔야 겠습니다.적당히 소비자 눈 속임 하십..곤란하죠...=========================='],
       ['센틸리아크림 상품평이벤트가 이제야 생각나서요 11일 까지던데... 못받는건가요?'],
       [ '의자는  배송을  안해주셨어요배송하시는분이  몰랐다고  하시면서   보내주신다고  하셨는데  그냥 넘어가면까먹고  배송  안해주실까봐  적어봅니다']], dtype=object)

In [36]:
df_srcontents.values.tolist()

[['왜 싱글만 배송되었나요? 퀸은 언제오는지요.. 배송완료되지 않았는데.. 확인부탁드립니다.'],
 ['취소하고싶은데취소가 안되내요  결제하고바로취소하려고눌럿는데 안눌려요 취소요청부탁드려요'],
 ['연락이 한번도 없네요 ㅠㅠ 취소부탁드립니다취소 가능한가요?'],
 ['현금영수증010 4506 8361발급해주세요~'],
 ['저 이상품 매직딜로 결제 완료 된거맞는거죠? 예상 적립은 얼마이며 언제 적립되나요? 혹시 매직딜로 적용된거 아니라면 상품 취소해 주세요'],
 ['이거 동생이 사달라고 해서 주문햇는데오늘 매장서 다른제품삿다고 주문취소해달라고 하네요주문 취소부탁드려요'],
 ['상품재질이 으깨어지면서 뭉침현상이 있습니다'],
 ['1월달 카드할인행사 정보를 알고싶은데요,~답변 부탁 드립니다'],
 ['결제카드 변경 하려는데  지금 국민카드 변경하면 7프로 혜택을 받을 수 있나요?'],
 ['12월 16일 주문했는데 오늘 포장을 뜯어서 확인을 했어요..그레이로 주문을 했는데 색상이 잘 못 왔는데 교환이 가능한가요?오늘 박스를 뜯은거라 비닐포장은 그대로 있고요.. 사진 첨'],
 ['방송상품구입한건데17%쿠폰이벤트해당되는거맞나요??'],
 ['달서구송현동 1993-10뉴파크빌 주문한거 언제 와요??같은날 주문한 물 은 부산에 벌써 도착햇는데'],
 [nan],
 ['문화상품권으로 바지를 사려하는데 어떻게해야하나요?'],
 ['오늘 국민카드 할인이던데내일날짜랑 며칠동안 카드할인사 카드할인률좀 알수있을까요?'],
 ['실측사이즈가 너무작게표현되어물건 구매후 입어보니  상당히 크네요교환을 하길 원 하는데 사이즈가 없으니해결 방법을 알려주세요BE,66 재고 있나요?'],
 ['휴일에시킨거니취소되죠?★ 주문번호 808868807 ... 취소진행 해도 될까요  ?'],
 ['10분 전에 구매했습니다만,주문취소하고 싶습니다. 처리 후 연락부탁합니다. 2개입니다'],
 ['안녕하세요 사과관련 문의 드려요! 저 방송때 사과주스 한박스 추첨에 당첨됐다고 문자왔었는데요 언제 배

In [37]:
arrTemp = df_srcontents.values.tolist()

In [40]:
arrTemp[0][0]

'왜 싱글만 배송되었나요? 퀸은 언제오는지요.. 배송완료되지 않았는데.. 확인부탁드립니다.'

In [44]:
nouns = t.nouns(arrTemp[0][0])

In [47]:
from collections import Counter

count = Counter(nouns)
count

Counter({'배송': 2, '싱글': 1, '언제': 1, '완료': 1, '왜': 1, '퀸': 1, '확인': 1})

In [49]:
for n, c in count.most_common(5):
    print(n,c)

배송 2
왜 1
확인 1
퀸 1
언제 1


In [50]:
type(count)

collections.Counter

In [54]:
arrTemp[1]

['취소하고싶은데취소가 안되내요  결제하고바로취소하려고눌럿는데 안눌려요 취소요청부탁드려요']

In [56]:
len(arrTemp)

18422

In [89]:
tempContents = ''
arrContents = []

In [90]:
for row in arrTemp:
    if type(row[0]) == str:
        arrContents.append(row[0])

In [91]:
arrContents

['왜 싱글만 배송되었나요? 퀸은 언제오는지요.. 배송완료되지 않았는데.. 확인부탁드립니다.',
 '취소하고싶은데취소가 안되내요  결제하고바로취소하려고눌럿는데 안눌려요 취소요청부탁드려요',
 '연락이 한번도 없네요 ㅠㅠ 취소부탁드립니다취소 가능한가요?',
 '현금영수증010 4506 8361발급해주세요~',
 '저 이상품 매직딜로 결제 완료 된거맞는거죠? 예상 적립은 얼마이며 언제 적립되나요? 혹시 매직딜로 적용된거 아니라면 상품 취소해 주세요',
 '이거 동생이 사달라고 해서 주문햇는데오늘 매장서 다른제품삿다고 주문취소해달라고 하네요주문 취소부탁드려요',
 '상품재질이 으깨어지면서 뭉침현상이 있습니다',
 '1월달 카드할인행사 정보를 알고싶은데요,~답변 부탁 드립니다',
 '결제카드 변경 하려는데  지금 국민카드 변경하면 7프로 혜택을 받을 수 있나요?',
 '12월 16일 주문했는데 오늘 포장을 뜯어서 확인을 했어요..그레이로 주문을 했는데 색상이 잘 못 왔는데 교환이 가능한가요?오늘 박스를 뜯은거라 비닐포장은 그대로 있고요.. 사진 첨',
 '방송상품구입한건데17%쿠폰이벤트해당되는거맞나요??',
 '달서구송현동 1993-10뉴파크빌 주문한거 언제 와요??같은날 주문한 물 은 부산에 벌써 도착햇는데',
 '문화상품권으로 바지를 사려하는데 어떻게해야하나요?',
 '오늘 국민카드 할인이던데내일날짜랑 며칠동안 카드할인사 카드할인률좀 알수있을까요?',
 '실측사이즈가 너무작게표현되어물건 구매후 입어보니  상당히 크네요교환을 하길 원 하는데 사이즈가 없으니해결 방법을 알려주세요BE,66 재고 있나요?',
 '휴일에시킨거니취소되죠?★ 주문번호 808868807 ... 취소진행 해도 될까요  ?',
 '10분 전에 구매했습니다만,주문취소하고 싶습니다. 처리 후 연락부탁합니다. 2개입니다',
 '안녕하세요 사과관련 문의 드려요! 저 방송때 사과주스 한박스 추첨에 당첨됐다고 문자왔었는데요 언제 배송되나요? 아 그리구, 결제를 다른 카드로 하려고하는데 만약 결재수단 변경하'

In [92]:
tempContent = ' '.join(arrContents)

In [98]:
tempContent

'왜 싱글만 배송되었나요? 퀸은 언제오는지요.. 배송완료되지 않았는데.. 확인부탁드립니다. 취소하고싶은데취소가 안되내요  결제하고바로취소하려고눌럿는데 안눌려요 취소요청부탁드려요 연락이 한번도 없네요 ㅠㅠ 취소부탁드립니다취소 가능한가요? 현금영수증010 4506 8361발급해주세요~ 저 이상품 매직딜로 결제 완료 된거맞는거죠? 예상 적립은 얼마이며 언제 적립되나요? 혹시 매직딜로 적용된거 아니라면 상품 취소해 주세요 이거 동생이 사달라고 해서 주문햇는데오늘 매장서 다른제품삿다고 주문취소해달라고 하네요주문 취소부탁드려요 상품재질이 으깨어지면서 뭉침현상이 있습니다 1월달 카드할인행사 정보를 알고싶은데요,~답변 부탁 드립니다 결제카드 변경 하려는데  지금 국민카드 변경하면 7프로 혜택을 받을 수 있나요? 12월 16일 주문했는데 오늘 포장을 뜯어서 확인을 했어요..그레이로 주문을 했는데 색상이 잘 못 왔는데 교환이 가능한가요?오늘 박스를 뜯은거라 비닐포장은 그대로 있고요.. 사진 첨 방송상품구입한건데17%쿠폰이벤트해당되는거맞나요?? 달서구송현동 1993-10뉴파크빌 주문한거 언제 와요??같은날 주문한 물 은 부산에 벌써 도착햇는데 문화상품권으로 바지를 사려하는데 어떻게해야하나요? 오늘 국민카드 할인이던데내일날짜랑 며칠동안 카드할인사 카드할인률좀 알수있을까요? 실측사이즈가 너무작게표현되어물건 구매후 입어보니  상당히 크네요교환을 하길 원 하는데 사이즈가 없으니해결 방법을 알려주세요BE,66 재고 있나요? 휴일에시킨거니취소되죠?★ 주문번호 808868807 ... 취소진행 해도 될까요  ? 10분 전에 구매했습니다만,주문취소하고 싶습니다. 처리 후 연락부탁합니다. 2개입니다 안녕하세요 사과관련 문의 드려요! 저 방송때 사과주스 한박스 추첨에 당첨됐다고 문자왔었는데요 언제 배송되나요? 아 그리구, 결제를 다른 카드로 하려고하는데 만약 결재수단 변경하 미궁 장사랑~제품 언제쯤 입고되나요? 주문시 신한5프로 청구할인이었는데 앱에서 1월 9, 10일에 국민카드로 결재변경하면 7

In [99]:
nouns = t.nouns(tempContent)
nouns

['왜',
 '싱글',
 '배송',
 '퀸',
 '언제',
 '배송',
 '완료',
 '확인',
 '취소',
 '결제',
 '바로',
 '눌럿',
 '데',
 '안눌려',
 '취소',
 '요청',
 '연락',
 '한번',
 '취소',
 '취소',
 '현금영수증',
 '발급',
 '저',
 '상품',
 '매직',
 '딜로',
 '결제',
 '완료',
 '된거',
 '예상',
 '적립',
 '얼마',
 '언제',
 '적립',
 '혹시',
 '매직',
 '딜로',
 '상품',
 '거',
 '동생',
 '사',
 '달라',
 '주문',
 '햇',
 '데',
 '오늘',
 '매장',
 '다른',
 '제품',
 '삿',
 '고',
 '주문',
 '취소',
 '주문',
 '취소',
 '상품',
 '재질',
 '뭉침현상',
 '월달',
 '카드',
 '할인',
 '행사',
 '정보',
 '알',
 '답변',
 '부탁',
 '결제',
 '카드',
 '변경',
 '지금',
 '국민카드',
 '변경',
 '프로',
 '혜택',
 '수',
 '월',
 '일',
 '오늘',
 '포장',
 '확인',
 '그레이',
 '주문',
 '색상',
 '못',
 '교환',
 '오늘',
 '박스',
 '비닐',
 '포장',
 '그대로',
 '사진',
 '방송',
 '상품',
 '구입',
 '건데',
 '쿠폰',
 '이벤트',
 '달서구',
 '송현동',
 '뉴',
 '파크빌',
 '언제',
 '와',
 '날',
 '물',
 '은',
 '부산',
 '벌써',
 '도착',
 '햇',
 '데',
 '문화',
 '상품권',
 '바지',
 '오늘',
 '국민카드',
 '할인',
 '데',
 '내일',
 '날짜',
 '며칠',
 '동안',
 '카드',
 '할인',
 '사',
 '카드',
 '할인',
 '률좀',
 '실',
 '사이즈',
 '너무',
 '물건',
 '구매',
 '후',
 '교환',
 '원',
 '사이즈',
 '해결',
 '방법',
 '재고',
 '휴일',
 '취소',
 '주문',
 '번호',


In [100]:
count = Counter(nouns)
count

Counter({'성장': 3,
         '발효': 2,
         '개주': 1,
         '김은희': 1,
         '생필품': 3,
         '레이디스': 1,
         '뷰어': 1,
         '이후': 109,
         '보기': 25,
         '일리': 1,
         '뻑뻑': 5,
         '전화번호': 59,
         '고객센뎌': 1,
         '오휘': 1,
         '은희': 2,
         '사과문': 1,
         '셩': 1,
         '선사': 1,
         '국밥': 2,
         '주로': 4,
         '앞뒤': 3,
         '겸용': 3,
         '서로': 12,
         '어머니': 18,
         '가요': 1048,
         '해초': 1,
         '삼척': 1,
         '욧': 2,
         '김미경': 1,
         '개떡': 1,
         '란걸': 1,
         '스토케': 1,
         '철학': 1,
         '노인': 2,
         '보내즈세': 1,
         '이자': 33,
         '사기': 7,
         '파트': 2,
         '에이프릴': 1,
         '펌프': 24,
         '갯수': 11,
         '회식': 1,
         '다시': 739,
         '통안': 1,
         '용암동': 1,
         '마개': 6,
         '숲': 1,
         '성우': 2,
         '어차피': 10,
         '진천읍': 1,
         '풋': 5,
         '꼭낼': 1,
         '이성희': 1,
         '잔뜩': 

In [101]:
import pandas as pd
df = pd.DataFrame.from_dict(count, orient='index').reset_index()
df

,index,0
0,환뷸,1
1,그다지,1
2,월,793
3,성장,3
4,뽀로로,1
5,이화여대,1
6,강동구,3
7,남바,1
8,발효,2
9,박정자,1


In [102]:
df.to_excel('sr_count.xlsx', sheet_name='sr')